# Main code
This file contains the construction of a knowledge graph, entity resolution and evaluation of knowledge graph with RAG. 

## Constructing KG

In [ ]:
import asyncio
import os
from datetime import datetime
from hashlib import md5
from typing import List

from pydantic import BaseModel, Field

from langchain_community.graphs import Neo4jGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_openai import ChatOpenAI
from langchain_text_splitters import TokenTextSplitter

In [ ]:
os.environ["NEO4J_URI"] = os.getenv('NEO4J_URI')
os.environ["NEO4J_USERNAME"] = os.getenv('NEO4J_USERNAME')
os.environ["NEO4J_PASSWORD"] = os.getenv('NEO4J_PASSWORD')
os.environ["OPENAI_API_KEY"] =  os.getenv('OPENAI_API_KEY')
NEO4j_URI = os.getenv('NEO4J_URI')
NEO4j_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4j_PASSWORD = os.getenv('NEO4J_PASSWORD')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

graph = Neo4jGraph(url = NEO4j_URI, username=NEO4j_USERNAME, password = NEO4j_PASSWORD, refresh_schema=False)

graph.query("CREATE CONSTRAINT IF NOT EXISTS FOR (c:Chunk) REQUIRE c.id IS UNIQUE")
graph.query("CREATE CONSTRAINT IF NOT EXISTS FOR (c:AtomicFact) REQUIRE c.id IS UNIQUE")
graph.query("CREATE CONSTRAINT IF NOT EXISTS FOR (c:KeyElement) REQUIRE c.id IS UNIQUE")
graph.query("CREATE CONSTRAINT IF NOT EXISTS FOR (d:Document) REQUIRE d.id IS UNIQUE")

In [ ]:
# Define the folder path
folder_path = "data/arxiv_stat_text"
documents_full = []
document_names = []

# Loop through the files in the folder
for filename in os.listdir(folder_path):
    paper_path = os.path.join(folder_path, filename)
    # Check if it's a file
    if os.path.isfile(paper_path):
        document_names.append(filename)
        documents_full.append(open(paper_path).read())

## Construct system

Finetuning the prompt

In [ ]:
construction_system = """
You are an intelligent assistant tasked with extracting key elements and atomic facts from scientific articles to construct a knowledge graph. 
Your main goal is to identify the same concepts under different names.
Your output must be structured to identify entities, relationships, and hierarchical connections. 


### JSON Output Requirements:
- Ensure all output is valid JSON.
- Each atomic fact must include:
  1. `key_elements`: A **list of unique items** (equations, authors, concepts, methods, etc.). **Limit: 1 to 10 items.** Avoid duplicate or malformed items.
  2. `atomic_fact`: A **single, clear sentence** following the subject-predicate-object structure.
- Ensure that `key_elements` contains no invalid characters (e.g., newlines, excessive commas, or special characters). Remove redundant or irrelevant entries.


### KEY Extraction Guidelines:
1. **Entities**: Extract and classify essential nouns and phrases into the following categories:
   - **Authors**: Names of paper authors.
   - **Concepts**: Theories, definitions, or models introduced or discussed.
   - **Methods**: Experimental techniques, algorithms, or procedures.
   - **Findings**: Results, discoveries, or key conclusions.
   - **References**: Other papers, datasets, or sources cited. Full titles of other papers, datasets, or sources cited. Avoid vague references like "reference [4]" and replace them with the actual paper title or dataset name whenever possible. If the title is unavailable, include other identifying information (e.g., authors or publication year).
   - **Theorems**: Mathematical or logical propositions.
   - Cross-Document Entities: Highlight entities (authors, concepts, methods, etc.) that are shared with or similar to those in other documents.
   
2. **Relationships**: Identify and label verbs or phrases that connect the entities. Examples include:
   - **"Cites"**: Links an author or paper to a referenced work.
   - **"Proposes"**: Connects an author or paper to a method or concept.
   - **"Builds Upon"**: Indicates that a concept/method is an extension of prior work.
   - **"Validates"**: Links findings to methods or experiments.
   - Cross-Document Relationships: Identify relationships that connect entities across multiple documents (e.g., the same concept being expanded upon by different papers).

3. **Hierarchical Relationships**: Identify nested structures or dependencies, such as:
   - A theorem being part of a model.
   - A method comprising multiple steps or components.
   - Cross-document hierarchies, such as a concept from one paper being part of a larger framework in another.

4. **Atomic Facts**: Extract concise, indivisible facts with clear subject-predicate-object structure. Ensure each atomic fact aligns with the entities and relationships identified.

5. **Key Elements**: 
    - Ensure that there are no redundant or repeated key elements. 
    - If the same key element appears more than once, only include it once
    - If a key element is part of an equation, include the equation.

6. **Relevance and Commonality**:
   - Prioritize facts and relationships that are repeated across multiple papers.
   - Highlight connections between entities that are pivotal or query-worthy.
   - Emphasize shared or query-worthy entities and relationships between documents.

7. **Connecting Documents**:
When identifying entities, relationships, or references, always check for overlaps or connections with other documents (e.g., shared concepts, methods, authors, or references).
Explicitly note when:
    - An entity or relationship in the current document appears in or is similar to one from another document.
    - A finding validates or contrasts a finding from another paper.
    - A concept or method builds upon prior work from another document.

8. **Additional Guidelines**:
   - Replace pronouns with specific nouns (e.g., "it" becomes the actual method or concept).
   - Include any implicit causal or temporal relationships.
   - Limit each `key_elements` list to **10 items maximum**. If there are more than 10 entities, select the most relevant ones.
   - Present the key elements and atomic facts in the same language as the original text (e.g., English or Chinese).
   - Avoid including vague or redundant entries in `key_elements` such as `,`or `\n` or `n` or `k`.
   - Ensure that atomic facts are distinct and not repeated.

Example Output:
---
**Key Elements**:
- Authors: John Doe, Jane Smith
- Concepts: Quantum Entanglement, Bell's Theorem
- Methods: Double-slit experiment
- Findings: Violation of Bell's inequality
- References: Paper A (Einstein, 1935)
- Equations: ax=b
- Cross-Document Entities: Bell's Theorem (also discussed in Paper B), Quantum Entanglement (validated by Paper C)

**Atomic Facts**:
1. John Doe and Jane Smith authored the paper.
2. The paper proposes the concept of Quantum Entanglement.
3. Bell's Theorem builds upon Einstein's 1935 work (Paper A).
4. The Double-slit experiment validates Quantum Entanglement.
5. The findings show a violation of Bell's inequality.
6. Cross-Document Fact: Bell's Theorem is cited in Paper B and extended by Paper C.
7. Cross-Document Fact: Quantum Entanglement was experimentally validated in Paper C, supporting findings in this paper.

---
Your output should maintain this structure and be as detailed and accurate as possible.
"""

construction_human = """Use the given format to extract information from the 
following input: {input}"""

construction_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            construction_system,
        ),
        (
            "human",
            (
                "Use the given format to extract information from the "
                "following input: {input}"
            ),
        ),
    ]
)

In [ ]:
class AtomicFact(BaseModel):
    key_elements: List[str]= Field(description="""A list of essential entities (e.g., authors, theories, methods, findings) that are pivotal to the atomic fact.
        These entities should align with the ones described in the prompt such as Authors, Concepts, Methods, Findings, References, Theorems, 
        and Cross-Document Entities. Ensure that the key elements are relevant and comprehensive. """) #Max length: 500 characters.""")
    atomic_fact: str = Field(description="""The smallest, indivisible facts, presented as concise sentences. These include
        propositions, theories, existences, concepts, and implicit elements like logic, causality, event
        sequences, interpersonal relationships, timelines, etc.""")

class Extraction(BaseModel):
    atomic_facts: List[AtomicFact] = Field(description="List of atomic facts")

## Model

In [ ]:
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.1,  
    check_every_n_seconds=0.1,
    max_bucket_size=10,  
)

model = ChatOpenAI(model="gpt-4o", temperature=0.1, rate_limiter=rate_limiter)
structured_llm = model.with_structured_output(Extraction)

construction_chain = construction_prompt | structured_llm

In [ ]:
import_query = """
MERGE (d:Document {id:$document_name})
WITH d
UNWIND $data AS row
MERGE (c:Chunk {id: row.chunk_id})
SET c.text = row.chunk_text,
    c.index = row.index,
    c.document_name = row.document_name
MERGE (d)-[:HAS_CHUNK]->(c)
WITH c, row
UNWIND row.atomic_facts AS af
MERGE (a:AtomicFact {id: af.id})
SET a.text = af.atomic_fact
MERGE (c)-[:HAS_ATOMIC_FACT]->(a)
WITH c, a, af
UNWIND af.key_elements AS ke
MERGE (k:KeyElement {id: ke})
MERGE (a)-[:HAS_KEY_ELEMENT]->(k)
"""

def encode_md5(text):
    return md5(text.encode("utf-8")).hexdigest()

In [ ]:
# Paper used 2k token size
async def process_document(text, document_name, chunk_size=2000, chunk_overlap=200):
    start = datetime.now()
    print(f"Started extraction at: {start}")
    text_splitter = TokenTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    texts = text_splitter.split_text(text)
    print(f"Total text chunks: {len(texts)}")
    tasks = [
        asyncio.create_task(construction_chain.ainvoke({"input":chunk_text}))
        for index, chunk_text in enumerate(texts)
    ]
    results = await asyncio.gather(*tasks)
    print(f"Finished LLM extraction after: {datetime.now() - start}")
    docs = [el.dict() for el in results]
    for index, doc in enumerate(docs):
        doc['chunk_id'] = encode_md5(texts[index])
        doc['chunk_text'] = texts[index]
        doc['index'] = index
        for af in doc["atomic_facts"]:
            af["id"] = encode_md5(af["atomic_fact"])
    # Import chunks/atomic facts/key elements
    graph.query(import_query, 
            params={"data": docs, "document_name": document_name})
    # Create next relationships between chunks
    graph.query("""MATCH (c:Chunk)<-[:HAS_CHUNK]-(d:Document)
WHERE d.id = $document_name
WITH c ORDER BY c.index WITH collect(c) AS nodes
UNWIND range(0, size(nodes) -2) AS index
WITH nodes[index] AS start, nodes[index + 1] AS end
MERGE (start)-[:NEXT]->(end)
""",
           params={"document_name":document_name})
    print(f"Finished import at: {datetime.now() - start}")

In [ ]:
def clean_graph():
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    graph.query(query)
# clean_graph()

### Insert each paper in KG

In [ ]:
local =["1505.04215v1.pdf.json",
        "2204.10909v2.pdf.json",
        "2209.01679v3.pdf.json",
        "1502.02355v2.pdf.json",
        "1812.00492v1.pdf.json",
        "1508.02925v1.pdf.json",
        "1605.04055v1.pdf.json",
        "1108.1098v1.pdf.json",
        "1611.04701v2.pdf.json",
        "2201.01879v3.pdf.json",
        "1504.05781v2.pdf.json"]

done = []

for text, name in zip(documents_full, document_names):
    if name in local and name not in done:
        print(name)
        await process_document(text, name, chunk_size=500, chunk_overlap=100)

## Entity resolution

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Optional

from retry import retry
from tqdm import tqdm

from graphdatascience import GraphDataScience
from neo4j import GraphDatabase

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector

#Neo4j connection
driver = GraphDatabase.driver(NEO4j_URI, auth=(NEO4j_USERNAME, NEO4j_PASSWORD))

### Generate embeddings for key elements

In [ ]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
    url=NEO4j_URI,
    username=NEO4j_USERNAME,
    password= NEO4j_PASSWORD,
    node_label="KeyElement", #["Document","Chunk", "AtomicFact", "KeyElement"],
    text_node_properties=["id"],#["id", "text", "index"], #['name', 'description', 'status'],
    embedding_node_property="embedding", #'embedding'
)

### Graph data science set up

In [ ]:
# project graph
gds = GraphDataScience(
    NEO4j_URI,
    auth=(NEO4j_USERNAME,NEO4j_PASSWORD)
)

G, result = gds.graph.project(
    "entities",                   #  Graph name
    "KeyElement",                 #  Node projection
    "*",                          #  Relationship projection
    nodeProperties=["embedding"]  #  Configuration parameters
)

### k-nearest neighbors similarity search using the embedding property

In [ ]:
similarity_threshold = 0.95

gds.knn.mutate(
  G,
  nodeProperties=['embedding'],
  mutateRelationshipType= 'SIMILAR',
  mutateProperty= 'score',
  similarityCutoff=similarity_threshold
)

In [ ]:
gds.wcc.write(
    G,
    writeProperty="wcc",
    relationshipTypes=["SIMILAR"]
)

### Identifying Duplicate Entities

In [ ]:
word_edit_distance = 3
potential_duplicate_candidates = graph.query(
    """MATCH (e:`KeyElement`)
    WHERE size(e.id) > 4 // longer than 4 characters
    WITH e.wcc AS community, collect(e) AS nodes, count(*) AS count
    WHERE count > 1
    UNWIND nodes AS node
    // Add text distance
    WITH distinct
      [n IN nodes WHERE apoc.text.distance(toLower(node.id), toLower(n.id)) < $distance | n.id] AS intermediate_results
    WHERE size(intermediate_results) > 1
    WITH collect(intermediate_results) AS results
    // combine groups together if they share elements
    UNWIND range(0, size(results)-1, 1) as index
    WITH results, index, results[index] as result
    WITH apoc.coll.sort(reduce(acc = result, index2 IN range(0, size(results)-1, 1) |
            CASE WHEN index <> index2 AND
                size(apoc.coll.intersection(acc, results[index2])) > 0
                THEN apoc.coll.union(acc, results[index2])
                ELSE acc
            END
    )) as combinedResult
    WITH distinct(combinedResult) as combinedResult
    // extra filtering
    WITH collect(combinedResult) as allCombinedResults
    UNWIND range(0, size(allCombinedResults)-1, 1) as combinedResultIndex
    WITH allCombinedResults[combinedResultIndex] as combinedResult, combinedResultIndex, allCombinedResults
    WHERE NOT any(x IN range(0,size(allCombinedResults)-1,1)
        WHERE x <> combinedResultIndex
        AND apoc.coll.containsAll(allCombinedResults[x], combinedResult)
    )
    RETURN combinedResult
    """, params={'distance': word_edit_distance})
potential_duplicate_candidates[:5]

### Processing Duplicate Entities with LM

In [ ]:
system_prompt = """You are a data processing assistant. Your task is to identify duplicate entities in a list and decide which of them should be merged.
The entities might be slightly different in format or content, but essentially refer to the same thing. Use your analytical skills to determine duplicates.

Here are the rules for identifying duplicates:
1. Entities with minor typographical differences should be considered duplicates.
2. Refrain from merging entities based on equations, as they may have different meanings.
3. Entities that refer to the same real-world object or concept, even if described differently, should be considered duplicates.
4. If it refers to different numbers, dates, or products, do not merge results
5. Mathematical equations must be identical to be considered duplicates.
"""
user_template = """
Here is the list of entities to process:
{entities}

Please identify duplicates, merge them, and provide the merged list.
"""

In [ ]:
class DuplicateEntities(BaseModel):
    entities: List[str] = Field(
        description="Entities that represent the same object or real-world entity and should be merged"
    )


class Disambiguate(BaseModel):
    merge_entities: Optional[List[DuplicateEntities]] = Field(
        description="Lists of entities that represent the same object or real-world entity and should be merged"
    )

extraction_llm = ChatOpenAI(model_name="gpt-4o-mini").with_structured_output(
    Disambiguate
)

extraction_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        (
            "human",
            user_template,
        ),
    ]
)

In [ ]:
extraction_chain = extraction_prompt | extraction_llm

@retry(tries=3, delay=2)
def entity_resolution(entities: List[str]) -> Optional[List[str]]:
    return [
        el.entities
        for el in extraction_chain.invoke({"entities": entities}).merge_entities
    ]

In [ ]:
MAX_WORKERS = 4

merged_entities = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Submitting all tasks and creating a list of future objects
    futures = [
        executor.submit(entity_resolution, el['combinedResult'])
        for el in potential_duplicate_candidates
    ]

    for future in tqdm(
        as_completed(futures), total=len(futures), desc="Processing documents"
    ):
        try:
            to_merge = future.result()
            # Ensure to_merge is not None and is iterable
            if to_merge:  # this checks if it's not None or an empty value
                merged_entities.extend(to_merge)
        except Exception as e:
            # Handle any exception raised during the processing of a future
            print(f"Error processing future: {e}")

### Merge entities in the graph using query

In [ ]:
# break down merged entities into chunks of 100
merged_entities_chunks = [merged_entities[i:i + 100] for i in range(0, len(merged_entities), 100)]


for i, chunk in enumerate(merged_entities_chunks):
  try:
    graph.query("""
    UNWIND $data AS candidates
    CALL {
      WITH candidates
      MATCH (e:KeyElement) WHERE e.id IN candidates
      RETURN collect(e) AS nodes
    }
    CALL apoc.refactor.mergeNodes(nodes, {properties: {
        description:'combine',
        `.*`: 'discard'
    }})
    YIELD node
    RETURN count(*)
    """, params={"data": chunk})
  except Exception as e:
    print(e)
    print("failed to merge chunk", i)


### Inspect graph

In [ ]:
graph.query("""MATCH (e) RETURN count(e)""")

In [ ]:
graph.query("""MATCH (e:KeyElement) RETURN count(e)""")

In [ ]:
G

In [ ]:
gds.graph.export(G, dbName = "thisisauniquename")

In [ ]:
gds.graph.get('entities')

## RAG for evaluation

In [1]:
from langchain.chains import GraphCypherQAChain
from langchain_core.prompts.prompt import PromptTemplate

/Users/albertebaht/Desktop/DTU/9_semester/02456 Deep learning/02456-deeplearning-project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use the embedding property for KeyElements
Only look for terms in KeyElements nodes
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Respond only with the Cypher query.

The question is:
{question} 

Important: In the generated Cypher query, look for terms in keyelements when using WHERE. 
In the generated Cypher query, the RETURN statement must explicitly include the property values used in the query's filtering condition but never the embedding property, alongside the main information requested from the original question. 
Do not return anything related to chunks node types. When using atomic facts the text property is important.

"""
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)


In [ ]:
graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_prompt = CYPHER_GENERATION_PROMPT,
    cypher_llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini"),
    qa_llm = ChatOpenAI(temperature=0,model_name="gpt-4o-mini"), graph=graph, verbose=True, allow_dangerous_requests=True
)

### Questions for Search agent

In [ ]:
cypher_chain.invoke(
    {"query": "Which document has most connections to other documents?"}
)

In [ ]:
response = cypher_chain.invoke({"query": "List `Theorem`s in the papers"})
print(response)

In [ ]:
response = cypher_chain.invoke({"query": "Explain the matematical detail of OLS"})
print(response)

In [ ]:
response = cypher_chain.invoke({"query": "Which paper uses WIDEHIST algorithm?"})
print(response)